In [1]:
import pandas as pd
import ast
import datasets
all  = pd.read_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/all_review_points.csv')

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(all)

207187

In [3]:
all.head()

,paper_id,venue,focused_review,point
0,ACL_2017_614_review,ACL_2017,- I don't understand effectiveness of the mult...,- I don't understand effectiveness of the mult...
1,ACL_2017_614_review,ACL_2017,- I don't understand effectiveness of the mult...,- The paper is not fully clear on a first read...
2,ACL_2017_614_review,ACL_2017,- I don't understand effectiveness of the mult...,- The relatively poor performance on nouns mak...
3,ACL_2017_614_review,ACL_2017,- I don't understand effectiveness of the mult...,-General Discussion: The paper is mostly strai...
4,ACL_2017_614_review,ACL_2017,- I don't understand effectiveness of the mult...,- Lines 367-368 : Why does X^{P} need to be sy...


In [ ]:
# Total number of samples
total_samples = 1200

# Uniform sampling
grouped = all.groupby("venue")
samples_per_group = max(1, total_samples // grouped.ngroups)  # Divide total_samples evenly
sampled_df = (
    grouped.apply(lambda group: group.sample(n=min(len(group), samples_per_group), random_state=42))
    .reset_index(drop=True)
)

# If more rows are still needed due to rounding, sample the remainder
remainder = total_samples - len(sampled_df)
if remainder > 0:
    remaining_sample = all.loc[~all.index.isin(sampled_df.index)].sample(n=remainder, random_state=42)
    sampled_df = pd.concat([sampled_df, remaining_sample]).reset_index(drop=True)

/tmp/ipykernel_692970/2537484748.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(lambda group: group.sample(n=min(len(group), samples_per_group), random_state=42))


In [5]:
sampled_df.value_counts("venue")

venue
ACL_2017      80
ARR_2022      80
EMNLP_2023    80
ICLR_2021     80
ICLR_2022     80
ICLR_2023     80
ICLR_2024     80
ICLR_2025     80
NIPS_2016     80
NIPS_2017     80
NIPS_2018     80
NIPS_2019     80
NIPS_2020     80
NIPS_2021     80
NIPS_2022     80
Name: count, dtype: int64

In [6]:
sampled_df.to_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_sampled.csv', index=False)

In [6]:
## add columns: actionability	grounding_specificity	verifiability	helpfulness	appropriateness	Do you need full review?
sampled_df["actionability"] = ""
sampled_df["grounding_specificity"] = ""
sampled_df["verifiability"] = ""
sampled_df["helpfulness"] = ""
sampled_df["appropriateness"] = ""
sampled_df["Do you need full review?"] = ""


In [12]:
## load HF authentication token from .env file
import os
from dotenv import load_dotenv
load_dotenv()
hf_token = os.getenv("HF_TOKEN")


In [14]:
# Convert DataFrame to Dataset
ds = datasets.Dataset.from_pandas(sampled_df)

# Push the dataset to the Hugging Face Hub
ds.push_to_hub("boda/review_rewrite_test",token=hf_token)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/boda/review_rewrite_test/commit/f5ed869dd440f121408094446dbc4f6929e170af', commit_message='Upload dataset', commit_description='', oid='f5ed869dd440f121408094446dbc4f6929e170af', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
# sampled_df.to_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/100_point_for_manual_annotation.csv', index=False)